In [1]:
import numpy as np
import matplotlib.pyplot as plt
from skimage import color
from skimage.feature import hog
from sklearn import svm
from sklearn import metrics
from tensorflow.keras.datasets.cifar10 import load_data
import cv2

In [2]:
(x_train, y_train), (x_test, y_test) = load_data()

In [3]:
x_train_hog_images = []
x_train_hog_features = []
for img in x_train:
    fd, hog_image = hog(img, orientations=8, pixels_per_cell=(4, 4), cells_per_block=(2, 2), block_norm='L2', visualize=True, channel_axis=-1)
    x_train_hog_images.append(hog_image)
    x_train_hog_features.append(fd)

In [4]:
x_test_hog_images = []
x_test_hog_features = []
for img in x_test:
    fd, hog_image = hog(img, orientations=8, pixels_per_cell=(4, 4), cells_per_block=(2, 2), block_norm='L2', visualize=True, channel_axis=-1)
    x_test_hog_images.append(hog_image)
    x_test_hog_features.append(fd)

In [5]:
x_train_hog_features, x_test_hog_features = np.array(x_train_hog_features), np.array(x_test_hog_features)
y_train, y_test = y_train.reshape(-1), y_test.reshape(-1)

In [8]:
svc = svm.SVC()
svc.fit(x_train_hog_features, y_train)

SVC()

In [27]:
y_pred = svc.predict(x_test_hog_features)

In [36]:
print('Accurancy:', metrics.accuracy_score(y_test, y_pred))
print('Precision:', metrics.precision_score(y_test, y_pred, average='weighted'))
print('Recall:', metrics.recall_score(y_test, y_pred, average='weighted'))
print('F1:', metrics.f1_score(y_test, y_pred, average='weighted'))

Accurancy: 0.6521
Precision: 0.6523939759923129
Recall: 0.6521
F1: 0.6516225582902705


In [6]:
x_train_gray, x_test_gray = [cv2.cvtColor(img, cv2.COLOR_RGB2GRAY) for img in x_train], [cv2.cvtColor(img, cv2.COLOR_RGB2GRAY) for img in x_test]

In [7]:
sift = cv2.SIFT_create()

In [8]:
x_train_sift_features = [sift.detectAndCompute(img, None)[1] for img in x_train_gray]
x_test_sift_features = [sift.detectAndCompute(img, None)[1] for img in x_test_gray]

In [9]:
x_train_sift_features_combined = []
y_train_sift_featured_extended = []
for i in range(len(x_train_sift_features)):
    if x_train_sift_features[i] is not None:
        for j in x_train_sift_features[i]:
            x_train_sift_features_combined.append(j)
            y_train_sift_featured_extended.append(y_train[i])

In [10]:
x_test_sift_features_combined = []
y_test_sift_featured_extended = []
for i in range(len(x_test_sift_features)):
    if x_test_sift_features[i] is not None:
        for j in x_test_sift_features[i]:
            x_test_sift_features_combined.append(j)
            y_test_sift_featured_extended.append(y_test[i])

In [11]:
x_train_sift_features_combined = np.array(x_train_sift_features_combined)
y_train_sift_featured_extended = np.array(y_train_sift_featured_extended)
x_test_sift_features_combined = np.array(x_test_sift_features_combined)
y_test_sift_featured_extended = np.array(y_test_sift_featured_extended)

In [ ]:
sift_svc = svm.SVC()
sift_svc.fit(x_train_sift_features_combined, y_train_sift_featured_extended)

In [12]:
x_train_sift_features_combined.shape

(656497, 128)

### Відібравши всі унікальні точки з зображень за допомогою методу SIFT вийшло 656497 унікальних точок та 128 ознак кожна. Після чого передавши всі записи на SVC. Метод обраховує наявні дані дуже довго щоб бути доцільним у використанні

In [13]:
from sklearn.tree import DecisionTreeClassifier

In [14]:
sift_dec_tree = DecisionTreeClassifier()
sift_dec_tree.fit(x_train_sift_features_combined, y_train_sift_featured_extended)

DecisionTreeClassifier()

In [15]:
sift_y_pred = sift_dec_tree.predict(x_test_sift_features_combined)

In [16]:
print('Accurancy:', metrics.accuracy_score(y_test_sift_featured_extended, sift_y_pred))
print('Precision:', metrics.precision_score(y_test_sift_featured_extended, sift_y_pred, average='weighted'))
print('Recall:', metrics.recall_score(y_test_sift_featured_extended, sift_y_pred, average='weighted'))
print('F1:', metrics.f1_score(y_test_sift_featured_extended, sift_y_pred, average='weighted'))

Accurancy: 0.14174395621103847
Precision: 0.14169140792704887
Recall: 0.14174395621103847
F1: 0.141709761246481


In [17]:
hog_dec_tree = DecisionTreeClassifier()
hog_dec_tree.fit(x_train_hog_features, y_train)

DecisionTreeClassifier()

In [18]:
hog_y_pred = hog_dec_tree.predict(x_test_hog_features)

In [19]:
print('Accurancy:', metrics.accuracy_score(y_test, hog_y_pred))
print('Precision:', metrics.precision_score(y_test, hog_y_pred, average='weighted'))
print('Recall:', metrics.recall_score(y_test, hog_y_pred, average='weighted'))
print('F1:', metrics.f1_score(y_test, hog_y_pred, average='weighted'))

Accurancy: 0.2278
Precision: 0.22756386617335478
Recall: 0.2278
F1: 0.22762907986235936


### Дерева рішень працюють набагато швидше для hog і sift, але метрики набагато гірші